# StraddleHedgeLoopEA.mq5

**Auto straddle + hedge EA for MT5**
This EA automatically places a straddle of pending orders, hedges positions on take-profit or stop-loss, manages trailing stops, cancels solo pending orders, and loops indefinitely. It is written purely with **MT5 functions** — no MT4-style functions are used. The EA also calculates position size based on **1% account risk per trade**.

---

## Features

1. **Automatic Straddle Orders**

   * Places **Buy Stop** above the current price.
   * Places **Sell Stop** below the current price.
   * Pending orders have individually calculated **stop-loss (SL) and take-profit (TP)** levels.

2. **Risk Management**

   * Calculates **lot size** based on 1% account equity risk.
   * Ensures lot size respects broker's **minimum, maximum, and step**.

3. **Hedging**

   * When a pending order triggers:

     * Cancels the opposite pending order.
   * Rebuilds straddle if both positions are closed.

4. **Trailing Stop**

   * Activates after a configurable profit in pips (`InpTrailTriggerPips`).
   * Moves stop-loss to lock in profits while maintaining a fixed trailing distance (`InpTrailDistancePips`).
   * Prevents stop-loss from moving backward.

5. **Solo Pending Order Protection**

   * Cancels a pending order if it is left alone without its hedge pair.

---

## Input Parameters

| Parameter              | Type   | Default | Description                                                        |
| ---------------------- | ------ | ------- | ------------------------------------------------------------------ |
| `InpRiskPercent`       | double | 1.0     | Percent of account equity to risk per trade.                       |
| `InpPipsEntry`         | int    | 100     | Distance in pips from current price to place Buy/Sell Stop.        |
| `InpPipsSL`            | int    | 100     | Stop-loss distance in pips.                                        |
| `InpPipsTP`            | int    | 200     | Take-profit distance in pips.                                      |
| `InpSlippage`          | int    | 10      | Maximum allowed slippage in points.                                |
| `InpMagic`             | ulong  | 7777    | Magic number to identify EA’s trades.                              |
| `InpTrailTriggerPips`  | int    | 50      | Profit in pips required before trailing stop activates.            |
| `InpTrailDistancePips` | int    | 40      | Distance in pips to maintain from current price for trailing stop. |

---

## Core Functions

### 1. `Pip()`

Calculates the pip size based on the symbol’s decimal digits. Handles 3/5 digit symbols correctly.

---

### 2. `CalculateLotSize(double stopLossPips)`

Calculates lot size based on the desired account risk percentage and the stop-loss in pips.

* Ensures lot respects broker min/max and step size.
* Returns the calculated lot size.

---

### 3. `PlacePending(ENUM_ORDER_TYPE type, double price, double sl, double tp, ulong &ticket, string comment)`

Places a pending order (Buy Stop or Sell Stop) with the correct lot size.

* Updates the order ticket if successfully placed.
* Prints error message if order fails.

---

### 4. `PlaceStraddle()`

Places a **Buy Stop** above and a **Sell Stop** below the current price.

* Calculates entry, SL, and TP based on `InpPipsEntry`, `InpPipsSL`, and `InpPipsTP`.
* Stores the pending order tickets in `buyStopTicket` and `sellStopTicket`.

---

### 5. `PendingOrderExists(ulong ticket)`

Checks if a pending order with a specific ticket exists. Returns `true` if exists, otherwise `false`.

---

### 6. `PositionExists(string type)`

Checks if a position exists for a given type ("BUY" or "SELL") with the EA’s magic number. Returns `true` if the position exists.

---

### 7. `CancelOrder(ulong &ticket)`

Cancels a pending order by ticket number.

* Resets ticket to `0` if successful.
* Prints status messages.

---

### 8. `CheckAndRebuild()`

Main logic loop:

* Cancels opposite pending order if one position is active.
* Rebuilds straddle if both positions are closed.
* Cancels solo pending orders to prevent orphan orders.

---

### 9. `TrailStops()`

Manages trailing stops:

* Iterates through all positions of the EA.
* Moves SL in the direction of profit once profit threshold is reached.
* Only moves SL forward (never backward).

---

## Event Handlers

### `OnInit()`

* Places initial straddle on EA startup.

### `OnTick()`

* Executes each tick:

  1. Checks positions and pending orders, rebuilds straddle if needed.
  2. Updates trailing stops.

---

## How It Works

1. On EA initialization, a straddle of Buy Stop and Sell Stop orders is placed relative to the current price.
2. When a pending order is triggered:

   * The opposite pending order is cancelled.
   * Trailing stops are managed according to profit thresholds.
3. Once both positions are closed, a new straddle is automatically placed.
4. Solo pending orders (without a hedge pair) are cancelled automatically.

---

## Notes

* Designed for **1-minute timeframe** or higher.
* Works on **all symbol types** supported by MT5.
* Fully MT5-native; avoids MT4 legacy functions.
* Risk management strictly follows 1% account risk per trade.


In [ ]:
//+------------------------------------------------------------------+
//|               StraddleHedgeLoopEA.mq5                            |
//|   Auto straddle + hedge on TP/SL, loops indefinitely             |
//|   Pure MT5 functions, no MT4-style functions                    |
//|   Trailing stop added & cancel solo pending orders               |
//|   Position sizing: risk 1% of account per order                  |
//+------------------------------------------------------------------+
#property strict
#include <Trade/Trade.mqh>

input double InpRiskPercent = 1.0; // Risk percent per trade
input int    InpPipsEntry = 100;
input int    InpPipsSL    = 100;
input int    InpPipsTP    = 200;
input int    InpSlippage  = 10;
input ulong  InpMagic     = 7777;
input int    InpTrailTriggerPips = 50;  // Profit pips to start trailing
input int    InpTrailDistancePips = 40; // Trailing distance

CTrade trade;

ulong buyStopTicket  = 0;
ulong sellStopTicket = 0;

//+------------------------------------------------------------------+
double Pip()
{
    double point  = SymbolInfoDouble(_Symbol, SYMBOL_POINT);
    int digits    = (int)SymbolInfoInteger(_Symbol, SYMBOL_DIGITS);
    return (digits == 3 || digits == 5) ? point * 10.0 : point;
}
//+------------------------------------------------------------------+
double CalculateLotSize(double stopLossPips)
{
    double equity = AccountInfoDouble(ACCOUNT_EQUITY);
    double pipValue = SymbolInfoDouble(_Symbol, SYMBOL_TRADE_TICK_VALUE);
    double lotStep = SymbolInfoDouble(_Symbol, SYMBOL_VOLUME_STEP);
    double minLot  = SymbolInfoDouble(_Symbol, SYMBOL_VOLUME_MIN);
    double maxLot  = SymbolInfoDouble(_Symbol, SYMBOL_VOLUME_MAX);

    // StopLoss in money
    double slMoney = stopLossPips * pipValue;

    double riskMoney = equity * InpRiskPercent / 100.0;

    double lots = riskMoney / slMoney;

    // Normalize lot to step & min/max
    lots = MathFloor(lots / lotStep) * lotStep;
    if(lots < minLot) lots = minLot;
    if(lots > maxLot) lots = maxLot;

    return(lots);
}
//+------------------------------------------------------------------+
bool PlacePending(ENUM_ORDER_TYPE type, double price, double sl, double tp, ulong &ticket, string comment)
{
    double stopLossPips = (type == ORDER_TYPE_BUY_STOP || type == ORDER_TYPE_BUY_LIMIT) ? (price - sl)/Pip()
                                                                                      : (sl - price)/Pip();
    double lots = CalculateLotSize(stopLossPips);

    MqlTradeRequest req;
    MqlTradeResult  res;
    ZeroMemory(req);
    ZeroMemory(res);

    req.action      = TRADE_ACTION_PENDING;
    req.symbol      = _Symbol;
    req.type        = type;
    req.volume      = lots;
    req.price       = price;
    req.sl          = sl;
    req.tp          = tp;
    req.magic       = InpMagic;
    req.deviation   = InpSlippage;
    req.type_time   = ORDER_TIME_GTC;
    req.type_filling= ORDER_FILLING_FOK;
    req.comment     = comment;

    if(!OrderSend(req, res))
    {
        Print("OrderSend failed: ", res.retcode, " ", res.comment);
        ticket = 0;
        return false;
    }

    ticket = res.order;
    return true;
}
//+------------------------------------------------------------------+
void PlaceStraddle()
{
    double pip   = Pip();
    double ask   = SymbolInfoDouble(_Symbol, SYMBOL_ASK);
    double bid   = SymbolInfoDouble(_Symbol, SYMBOL_BID);
    int digits   = (int)SymbolInfoInteger(_Symbol, SYMBOL_DIGITS);

    double buyE  = NormalizeDouble(ask + InpPipsEntry * pip, digits);
    double buySL = NormalizeDouble(buyE - InpPipsSL * pip, digits);
    double buyTP = NormalizeDouble(buyE + InpPipsTP * pip, digits);

    double sellE  = NormalizeDouble(bid - InpPipsEntry * pip, digits);
    double sellSL = NormalizeDouble(sellE + InpPipsSL * pip, digits);
    double sellTP = NormalizeDouble(sellE - InpPipsTP * pip, digits);

    PlacePending(ORDER_TYPE_BUY_STOP,  buyE,  buySL,  buyTP,  buyStopTicket,  "BUY_STOP_EA");
    PlacePending(ORDER_TYPE_SELL_STOP, sellE, sellSL, sellTP, sellStopTicket, "SELL_STOP_EA");
}
//+------------------------------------------------------------------+
bool PendingOrderExists(ulong ticket)
{
    if(ticket == 0) return false;

    if(OrderSelect(ticket))
        return true;

    return false;
}
//+------------------------------------------------------------------+
bool PositionExists(string type)
{
    for(int i = 0; i < PositionsTotal(); i++)
    {
        ulong ticket = PositionGetTicket(i);
        if(ticket == 0) continue;
        if(!PositionSelectByTicket(ticket)) continue;

        if(PositionGetInteger(POSITION_MAGIC) != (long)InpMagic) continue;
        if(PositionGetString(POSITION_SYMBOL) != _Symbol) continue;

        ENUM_POSITION_TYPE ptype = (ENUM_POSITION_TYPE)PositionGetInteger(POSITION_TYPE);
        if(type == "BUY"  && ptype == POSITION_TYPE_BUY)  return true;
        if(type == "SELL" && ptype == POSITION_TYPE_SELL) return true;
    }
    return false;
}
//+------------------------------------------------------------------+
void CancelOrder(ulong &ticket)
{
    if(ticket == 0) return;

    MqlTradeRequest req;
    MqlTradeResult  res;

    ZeroMemory(req);
    ZeroMemory(res);

    req.action = TRADE_ACTION_REMOVE;
    req.order  = ticket;
    req.symbol = _Symbol;

    if(OrderSend(req, res))
    {
        Print("Cancelled pending order: ", ticket);
        ticket = 0;
    }
    else
        Print("Failed to cancel order ", ticket, " ret=", res.retcode);
}
//+------------------------------------------------------------------+
void CheckAndRebuild()
{
    bool buyPos  = PositionExists("BUY");
    bool sellPos = PositionExists("SELL");

    bool buyActive  = PendingOrderExists(buyStopTicket);
    bool sellActive = PendingOrderExists(sellStopTicket);

    // One triggered → cancel opposite pending
    if(buyPos && sellActive)
        CancelOrder(sellStopTicket);
    if(sellPos && buyActive)
        CancelOrder(buyStopTicket);

    // Loop: if both positions closed → rebuild straddle
    if(!buyPos && !sellPos && !buyActive && !sellActive)
    {
        Print("Both positions closed → placing new straddle");
        PlaceStraddle();
    }

    // Check solo pending orders and cancel them
    if(buyActive && !sellActive)
        CancelOrder(buyStopTicket);
    if(sellActive && !buyActive)
        CancelOrder(sellStopTicket);
}
//+------------------------------------------------------------------+
void TrailStops()
{
    double pip = Pip();

    for(int i=0; i<PositionsTotal(); i++)
    {
        ulong ticket = PositionGetTicket(i);
        if(ticket == 0) continue;
        if(!PositionSelectByTicket(ticket)) continue;

        if(PositionGetInteger(POSITION_MAGIC) != (long)InpMagic) continue;
        if(PositionGetString(POSITION_SYMBOL) != _Symbol) continue;

        double open    = PositionGetDouble(POSITION_PRICE_OPEN);
        double current = (PositionGetInteger(POSITION_TYPE) == POSITION_TYPE_BUY) ? SymbolInfoDouble(_Symbol, SYMBOL_BID)
                                                                                  : SymbolInfoDouble(_Symbol, SYMBOL_ASK);
        double sl      = PositionGetDouble(POSITION_SL);
        ENUM_POSITION_TYPE type = (ENUM_POSITION_TYPE)PositionGetInteger(POSITION_TYPE);

        double profitPips = (type == POSITION_TYPE_BUY)  ? (current - open)/pip
                                                         : (open - current)/pip;

        if(profitPips >= InpTrailTriggerPips)
        {
            double newSL = (type == POSITION_TYPE_BUY)  ? current - InpTrailDistancePips*pip
                                                        : current + InpTrailDistancePips*pip;

            // Only move SL forward (do not move backward)
            if((type == POSITION_TYPE_BUY && newSL > sl) || (type == POSITION_TYPE_SELL && newSL < sl))
            {
                trade.PositionModify(ticket, newSL, PositionGetDouble(POSITION_TP));
                Print("Trailing SL modified for ticket ", ticket, " newSL=", newSL);
            }
        }
    }
}
//+------------------------------------------------------------------+
int OnInit()
{
    PlaceStraddle();
    return(INIT_SUCCEEDED);
}
//+------------------------------------------------------------------+
void OnTick()
{
    CheckAndRebuild();
    TrailStops();
}
//+------------------------------------------------------------------+
